In [1]:
import jax.numpy as np
import jax
from transformers import FlaxCLIPModel, AutoProcessor

import sys
sys.path.append("../")

from utils.dataset_utils import make_dataloader, create_input_iter
from utils.text_utils import process_truncate_captions, tokenize_captions

2023-12-30 21:37:43.039919: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-30 21:37:43.039985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-30 21:37:43.041322: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
batch_size = 2

files = ['../data/tfrecords_v4/observations_train_1.tfrecord','../data/tfrecords_v4/observations_train_2.tfrecord',]
ds = make_dataloader(files, batch_size=batch_size, seed=42)
batches = create_input_iter(ds)

In [3]:
model = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch16")

In [4]:
from models.clip_transfer import FlaxCLIPModuleTransfer

In [5]:
# model_frozen = FlaxCLIPModuleTransfer(model.config, d_head=1024)

In [6]:
from transformers.models.clip import FlaxCLIPPreTrainedModel

class FlaxCLIPModelTransfer(FlaxCLIPPreTrainedModel):
    module_class = FlaxCLIPModuleTransfer

model_transfer = FlaxCLIPModelTransfer(config=model.config)

In [13]:
def get_num_params(params):
    return sum(x.size for x in jax.tree_util.tree_leaves(params))

get_num_params(model_transfer.params), get_num_params(model.params)

(152245505, 149620737)

In [14]:
model_transfer.params['text_model']['text_backbone'] = model.params['text_model']
model_transfer.params['vision_model']['vision_backbone'] = model.params['vision_model']

model, params = model_transfer, model_transfer.params

In [ ]:
# from flax import traverse_util
# import optax

# partition_optimizers = {'trainable': optax.adam(5e-3), 'frozen': optax.set_to_zero()}
# param_partitions = traverse_util.path_aware_map(lambda path, v: 'frozen' if (('vision_backbone' in path) or ('text_backbone' in path)) else 'trainable', params)

# tx = optax.multi_transform(partition_optimizers, param_partitions)